# Calibration QAQC
- Visualize calibration routes and compare to the chosen and shortest routes
- Trip-specific impedance routing to see if chosen route can be found
- Test different objective functions
- Try using coordinates in case the map matched trace is incorrect

In [ ]:
from pathlib import Path
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
import networkx as nx
from stochopy.optimize import minimize
from tqdm import tqdm
import similaritymeasures
import random
import matplotlib.pyplot as plt
from shapely.ops import Point, MultiLineString, LineString
from importlib import reload
import datetime
from scipy.spatial.distance import directed_hausdorff
np.set_printoptions(suppress=True)

from bikewaysim.paths import config, stadia_toner, maptiler_streets
from bikewaysim.impedance_calibration import stochastic_optimization, speedfactor
from bikewaysim.network import modeling_turns
from bikewaysim.routing import rustworkx_routing_funcs
from bikewaysim.impedance_calibration import optimization_viz


from step_1_calibration_experiments import all_calibrations, full_model

# Import relevant files

In [ ]:
links, turns, length_dict, geo_dict, turn_G = rustworkx_routing_funcs.import_calibration_network(config)
with (config['calibration_fp']/'ready_for_calibration_stats.pkl').open('rb') as fh: # has loss values for shortest path
    full_set = pickle.load(fh)
full_ods = stochastic_optimization.match_results_to_ods_w_year(full_set)
# for adding coordinates to the viz part
with (config['cycleatl_fp']/"rdp.pkl").open('rb') as fh:
    coords = pickle.load(fh)

with (config['cycleatl_fp']/"trips_2.pkl").open('rb') as fh:
    trips = pickle.load(fh)
with (config['cycleatl_fp']/"users_2.pkl").open('rb') as fh:
    users = pickle.load(fh)
trips.set_index('tripid',inplace=True)
users.set_index('userid',inplace=True)
trips = trips.loc[list(full_set.keys())]
users = users.loc[users.index.isin(set(list(trips['userid'])))]
user_map = trips['userid'].to_dict()

# First an overview of how to use the QAQC functions

In [ ]:
# retrieves the calibration result for each model calibrated
calibration_results_fps = (config['calibration_fp']/'results').glob('*.pkl')
calibration_results = {}
for calibration_results_fp in calibration_results_fps:
    with calibration_results_fp.open('rb') as fh:
        calibration_results[calibration_results_fp.stem] = pickle.load(fh)

# retrieves post routing and loss values results for each model calibrated
post_calibration_fps = (config['calibration_fp']/'loss').glob('*.pkl')
post_calibration = {}
for post_calibration_fp in post_calibration_fps:
    with post_calibration_fp.open('rb') as fh:
        post_calibration[post_calibration_fp.stem] = pickle.load(fh)

## Visualize a random trip

In [ ]:
# get 
pool = list(full_set.keys())
not_in_pool = []
for key, post_calibration_dict in post_calibration.items():
    [x for x in pool if x not in post_calibration_dict.keys()]

In [ ]:
post_calibration.keys()

In [ ]:
#TODO add route attributes and the betas used for each line
reload(stochastic_optimization)

tripid = random.choice(pool)
# tripid = 2323
userid = user_map[tripid]
print(tripid,userid)
# tripid = 32136 not enough difference between streets to matter
reload(stochastic_optimization)
reload(optimization_viz)
# subset_post_calibration = {key:item for key,item in post_calibration.items() if user_map[tripid] == int(key.split('_')[0])}
print(trips.loc[tripid,['avg_speed_mph','trip_type','description']])
print(users.loc[trips.at[tripid,'userid']])
m = optimization_viz.visualize_three(tripid,full_set,post_calibration,calibration_results,geo_dict,coords,config['projected_crs_epsg'],stadia_toner)
m.save(Path.home()/'Downloads/test_viz.html')
m

## Try out custom impedance

In [ ]:
betas_tup = (
    {'col': '2lpd', 'type': 'link', 'range': [0, 3], 'beta': 2.8513},
    {'col': '3+lpd', 'type': 'link', 'range': [0, 3], 'beta': 5},
    {'col': '(30,40] mph', 'type': 'link', 'range': [0, 3], 'beta': 1.216},
    {'col': '(40,inf) mph', 'type': 'link', 'range': [0, 3], 'beta': 0.2833},
    {'col': '[4k,10k) aadt', 'type': 'link', 'range': [0, 3], 'beta': 1.4558},
    {'col': '[10k,inf) aadt', 'type': 'link', 'range': [0, 3], 'beta': 1.3963},
    {'col': '[4,6) grade', 'type': 'link', 'range': [0, 3], 'beta': 2.0827},
    {'col': '[6,inf) grade', 'type': 'link', 'range': [0, 3], 'beta': 1.4008},
    {'col': 'bike lane', 'type': 'link', 'range': [-1, 0], 'beta': -0.4116},
    {'col': 'cycletrack', 'type': 'link', 'range': [-1, 0], 'beta': -0.4478},
    {'col': 'multi use path', 'type': 'link', 'range': [-1, 0], 'beta': -0.3064},
    {'col': 'unsig_crossing', 'type': 'turn', 'range': [0, 2], 'beta': 0.1648}
)
custom_route = stochastic_optimization.create_custom_route(betas_tup,tripid,full_set,links,turns,turn_G,length_dict,geo_dict)
m = optimization_viz.visualize_three(tripid,full_set,post_calibration,calibration_results,geo_dict,coords,config['projected_crs_epsg'],stadia_toner,custom_route)
m

## Idenfity trips that pass through a circle
One area with trouble is the Stone Mountain Trail because the map matched route often uses Dekalb instead. This leads to some trips having a lower than expected overlap.

In [ ]:
reload(stochastic_optimization)

In [ ]:
freedom_pkwy = (2237899.09,1372338.05)
freedom_pkwy = stochastic_optimization.retrieve_geos(*freedom_pkwy,modeled_results,links)
print(len(freedom_pkwy))

In [ ]:
smt = (2250499.40,1369121.80)
smt = stochastic_optimization.retrieve_geos(*smt,modeled_results,links)
print(len(smt))
tenth_st = (2233722.10,1375729.08)
tenth_st = stochastic_optimization.retrieve_geos(*tenth_st,modeled_results,links)
print(len(tenth_st))
wylie_st = (2237751.33,1365098.89)
wylie_st = stochastic_optimization.retrieve_geos(*wylie_st,modeled_results,links)
print(len(wylie_st))

In [ ]:
tripid = random.choice(freedom_pkwy)
reload(stochastic_optimization)
stochastic_optimization.basic_three_viz(tripid,modeled_results,links.crs,length_dict,geo_dict,tile_info_dict)